In [1]:
import os
import cv2
import easyocr
import torch
import matplotlib.pyplot as plt
import time
%matplotlib inline

### Setup paths

In [2]:
ROOT = os.path.abspath("..")

PATHS = {
    "data": os.path.join(ROOT, "data"),
    "models": os.path.join(ROOT, "models"),
}
DATA = {
    "img00" : os.path.join(PATHS["data"], "car00.webp"),
    "img01" : os.path.join(PATHS["data"], "car01.webp"),
    "img02" : os.path.join(PATHS["data"], "car02.jpg"),
    "img03" : os.path.join(PATHS["data"], "car03.jpg"),
    "img04" : os.path.join(PATHS["data"], "car04.jpg"),
    "video" : os.path.join(PATHS["data"], "video.mp4"),
}

MODELS = {
    "plates_pt" : os.path.join(PATHS["models"], "plates.pt"),
    "plates_n_pt" : os.path.join(PATHS["models"], "plates_n.pt"),
    "plates_onnx" : os.path.join(PATHS["models"], "plates.onnx"),
    "platesv8_pt" : os.path.join(PATHS["models"], "plates_yolov8.pt"),
    "platesv8_onnx" : os.path.join(PATHS["models"], "plates_yolov8.onnx"),
    "platesv5n_onnx" : os.path.join(PATHS["models"], "plates_yolov5n.onnx"),
}

SELECTED_MODEL = MODELS["plates_n_pt"]
SELECTED_IMAGE = DATA["img01"]

In [ ]:
net = cv2.dnn.readNetFromONNX(MODELS["platesv5n_onnx"])
type(net)

In [ ]:
ln = net.getLayerNames()
input_layer_idx = net.getLayerId(ln[0])
net.getLayer(input_layer_idx)

In [ ]:
image = cv2.cvtColor(cv2.imread(SELECTED_IMAGE), cv2.COLOR_BGR2RGB)
height, width = image.shape[:2]
plt.imshow(image)
plt.show()

In [ ]:
blob = cv2.dnn.blobFromImage(image, 1/255, (640, 640), swapRB=False, crop=False)
net.setInput(blob)
outputs = net.forward()
outputs[0][0]

In [ ]:
outputs = outputs.transpose((0, 2, 1))
outputs.shape

In [ ]:
idx = outputs[0][:, 4] > 0.5
idx.sum()

In [ ]:
for row in outputs[0][idx]:
    print(row)

In [ ]:
detections = outputs[0][idx]
rect = image.copy()

for detection in detections:
    conf = outputs[0][idx][-2]
    x, y, w, h = detection[:4].astype(int)
    x1, y1 = x-w//2, y-h//2
    x2, y2 = x+w//2, y+h//2
    x1 = x1 * width // 640
    y1 = y1 * height // 640
    x2 = x2 * width // 640
    y2 = y2 * height // 640
    top_left = (x1, y1)
    bottom_right = (x2, y2)
    rect = cv2.rectangle(rect, top_left, bottom_right, (255, 0, 0), 5)

plt.imshow(rect)
plt.show()

In [ ]:
def getbboxv5(frame, width, height):
    blob = cv2.dnn.blobFromImage(frame, 1/255, (640, 640), swapRB=False, crop=False)
    net.setInput(blob)
    outputs = net.forward()
    idx = outputs[0][:, 4] > 0.80
    rect = frame.copy()
    detections = outputs[0][idx]
    
    for detection in detections:
        conf = detection[-2]
        x, y, w, h = detection[:4].astype(int)
        x1, y1 = x-w//2, y-h//2
        x2, y2 = x+w//2, y+h//2
        x1 = x1 * width // 640
        y1 = y1 * height // 640
        x2 = x2 * width // 640
        y2 = y2 * height // 640
        top_left = (x1, y1)
        bottom_right = (x2, y2)
        rect = cv2.rectangle(rect, top_left, bottom_right, (255, 0, 0), 5)
    
    return rect

In [ ]:
def getbboxv8(frame, width, height):
    blob = cv2.dnn.blobFromImage(frame, 1/255, (640, 640), swapRB=False, crop=False)
    net.setInput(blob)
    outputs = net.forward()
    outputs = outputs.transpose((0, 2, 1))
    idx = outputs[0][:, 4] > 0.50
    rect = frame.copy()
    detections = outputs[0][idx]
    
    for detection in detections:
        conf = detection[-2]
        x, y, w, h = detection[:4].astype(int)
        x1, y1 = x-w//2, y-h//2
        x2, y2 = x+w//2, y+h//2
        x1 = x1 * width // 640
        y1 = y1 * height // 640
        x2 = x2 * width // 640
        y2 = y2 * height // 640
        top_left = (x1, y1)
        bottom_right = (x2, y2)
        rect = cv2.rectangle(rect, top_left, bottom_right, (255, 0, 0), 5)
    
    return rect

In [ ]:
cap = cv2.VideoCapture(DATA["video"])
assert cap.isOpened()

prev_frame_time = 0
new_frame_time = 0

while (True):
    ret, frame = cap.read()

    if not ret:
        break
    
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    fps = int(fps)
    fps = str(fps)
    cv2.putText(frame, f"{fps} fps", (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)

    height, width = frame.shape[:2]

    rect = getbboxv5(frame, width, height)
    # rect = getbboxv8(frame, width, height)

    cv2.imshow("Plate Detection", rect)

    if (cv2.waitKey(5) & 0xFF == 27):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
from PlateDetector import PlateDetector
from Plate import Plate

In [4]:
pd = PlateDetector(MODELS["platesv5n_onnx"], 640)

cap = cv2.VideoCapture(DATA["video"])
assert cap.isOpened()

prev_frame_time = 0
new_frame_time = 0

while (True):
    ret, frame = cap.read()

    if not ret:
        break
    
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    fps = int(fps)
    fps = str(fps)
    cv2.putText(frame, f"{fps} fps", (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)

    plates = pd.Predict(frame)
    print(plates)

    if (cv2.waitKey(5) & 0xFF == 27):
        break

cap.release()
cv2.destroyAllWindows()

IndexError: invalid index to scalar variable.